#  Python Library

**Benjamin Corbett**  
April 23, 2025

## Import Libraries

In [1]:
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
import smtplib
import json
from itertools import zip_longest
from typing import List
import re
from deep_translator import GoogleTranslator

load_dotenv() 

True

## Abstract Base Class

In [4]:
class Periodical(ABC):
    def __init__(self, title, author, available=True):
        self.title = title
        self.author = author
        self.available = available
        
    @abstractmethod
    def display_info(self):
        pass

## Russian Book Class

In [7]:
class RussianBook:
    def __init__(self, russian_title, russian_author):
        self.russian_title = russian_title
        self.russian_author = russian_author

    def get_info(self):
        return f"{self.russian_title} by {self.russian_author}"

## Adapter Class

In [10]:
class RussianTranslatorAdapter(Periodical):
    def __init__(self, russian_book: RussianBook):
        
        translator = GoogleTranslator(source='ru', target='en')
        
        self.title = translator.translate(russian_book.russian_title)
        self.author = translator.translate(russian_book.russian_author)
        self.available = True
        self._original = russian_book

    def display_info(self):
        print(f"[Book - Translated] '{self.title}' by {self.author}")
        print(f"(Original: '{self._original.russian_title}' by {self._original.russian_author})")

    def display_info(self):
        print(f"[Book - Translated] '{self.title}' by {self.author}")
        print(f"(Original: '{self._original.russian_title}' by {self._original.russian_author}) \n")

## Subclasses

In [13]:
class Book(Periodical):
       
    def display_info(self):
        print(f"[Book] '{self.title} by {self.author}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

In [15]:
class Newspaper(Periodical):
    def __init__(self, title, author, available=True, publish_date=str):

        if not re.match(r"^(0[1-9]|1[0-2])/([0-9]{4})$", publish_date):
            raise ValueError("Date must be in MM/YYYY format")
            
        super().__init__(title, author, available)
        self.publish_date = publish_date
        
    def display_info(self):
        print(f"[Newspaper] '{self.title} by {self.author} Published: {self.publish_date}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

In [17]:
class Magazine(Periodical):
    def display_info(self):
        print(f"[Magazine] '{self.title} by {self.author}'")
        print(f"Available: {'Yes' if self.available else 'No'}")

## Library Class


In [20]:
class Library:
    def __init__(self):
        self.books: List[Periodical] = []

    def add_book(self, book: Periodical):
        self.books.append(book)
        print(f"Added {book.title} by {book.author} to library")

    def remove_book(self, title: str):
        for book in self.books:
            if book.title == title:
                self.books.remove(book)
                print(f"Removed {book.title} from library")
                return
        print("Not found.")

    def list_all(self) -> str:
        if not self.books:
            return "Your library is empty."
    
        result = ["Library:"]
        for book in self.books:
            periodical_type = book.__class__.__name__
            result.append(f"[{periodical_type}] Title: {book.title}")
            result.append(f"Author: {book.author}")
            
            if hasattr(book, "publish_date"):
                result.append(f"Published: {book.publish_date}")
            
            result.append(f"Available: {'Yes' if book.available else 'No'}")
            result.append("")  
        return '\n'.join(result)

    def remove_all_books(self):
        count = len(self.books)
        self.books.clear()
        print(f"Removed all {count} books from library")

    def to_json(self) -> str:
        """Returns the library as a JSON string"""
        data = [
            {
                "Title": book.title,
                "Author": book.author,
                "Available": "Yes" if book.available else "No"
            }
            for book in self.books
        ]
        return json.dumps(data, indent=2)  
       

## Book Scraping

In [23]:
url = "https://books.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
category_elements = soup.select('.side_categories ul li ul li a')

In [27]:
categories = []

In [29]:
for cat in category_elements:
    name = cat.get_text(strip=True)
    link = url + cat['href']
    categories.append({"name": name, "url": link})

## Categories

In [32]:
half = len(categories) // 2 + len(categories) % 2
left = categories[:half]
right = categories[half:]
for i, (left_cat, right_cat) in enumerate(zip_longest(left, right), start=1):
    left_str = f"{i}. {left_cat['name']}" if left_cat else ""
    right_str = f"{i + half}. {right_cat['name']}" if right_cat else ""
    print(f"{left_str:<35} {right_str}")

1. Travel                           26. Autobiography
2. Mystery                          27. Parenting
3. Historical Fiction               28. Adult Fiction
4. Sequential Art                   29. Humor
5. Classics                         30. Horror
6. Philosophy                       31. History
7. Romance                          32. Food and Drink
8. Womens Fiction                   33. Christian Fiction
9. Fiction                          34. Business
10. Childrens                       35. Biography
11. Religion                        36. Thriller
12. Nonfiction                      37. Contemporary
13. Music                           38. Spirituality
14. Default                         39. Academic
15. Science Fiction                 40. Self Help
16. Sports and Games                41. Historical
17. Add a comment                   42. Christian
18. Fantasy                         43. Suspense
19. New Adult                       44. Short Stories
20. Young Adult                

## User Selects Category From List

In [35]:
while True:
    try:
        choice = int(input(f"\nEnter the number 1 through {len(categories)} to select category you'd like to explore: "))
        if 1 <= choice <= len(categories):
            selected_category = categories[choice - 1]
            break
        else:
            print(f"Please enter a number between 1 and {len(categories)}.")
    except ValueError:
        print("Invalid input. Please enter a number.")



Enter the number 1 through 50 to select category you'd like to explore:  3


## Scrape Books based on Category Selection

Scrape books and collect book titles

In [38]:
cat_response = requests.get(selected_category["url"])
cat_soup = BeautifulSoup(cat_response.text, 'html.parser')

book_elements = cat_soup.select('article.product_pod')
titles = set() 
books = []

for book in book_elements:
    title = book.h3.a['title']
    if title not in titles:
        titles.add(title)
        books.append({"title": title})

In [40]:

for i, book in enumerate(books, start=1):
    print(f"{i}. {book['title']}")

1. Tipping the Velvet
2. Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton
3. A Flight of Arrows (The Pathfinders #2)
4. The House by the Lake
5. Mrs. Houdini
6. The Marriage of Opposites
7. Glory over Everything: Beyond The Kitchen House
8. Love, Lies and Spies
9. A Paris Apartment
10. Lilac Girls
11. The Constant Princess (The Tudor Court #1)
12. The Invention of Wings
13. World Without End (The Pillars of the Earth #2)
14. The Passion of Dolssa
15. Girl With a Pearl Earring
16. Voyager (Outlander #3)
17. The Red Tent
18. The Last Painting of Sara de Vos
19. The Guernsey Literary and Potato Peel Pie Society
20. Girl in the Blue Coat


## Client Adding To Their Library 

In [43]:
my_library = Library()

In [45]:
while True:
    user_input = input("\nEnter the corresponding number of the book you'd like to add: ")
    if user_input.isdigit():
        selection = int(user_input)
        if 1 <= selection <= len(books): 
            break
        else:
            print(f"Please enter a number between 1 and {len(books)}.")
    else:
        print("Invalid input. Please enter a number.")

selected_title = books[selection - 1]["title"]


Enter the corresponding number of the book you'd like to add:  30


Please enter a number between 1 and 20.



Enter the corresponding number of the book you'd like to add:  20


In [47]:
author_input = input(f"Who is the author of '{selected_title}'? ")

Who is the author of 'Girl in the Blue Coat'?  Jim Sm


In [49]:
new_book = Book(title=selected_title, author=author_input)
my_library.add_book(new_book)

Added Girl in the Blue Coat by Jim Sm to library


In [51]:
email_json = my_library.to_json()
print(email_json)

[
  {
    "Title": "Girl in the Blue Coat",
    "Author": "Jim Sm",
    "Available": "Yes"
  }
]


## Log In To Gmail and send the secret subject line to add the book to the library.

In [54]:
smtp_object = smtplib.SMTP('smtp.gmail.com',587)

In [55]:
smtp_object.ehlo()

(250,
 b'smtp.gmail.com at your service, [71.59.85.84]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [56]:
smtp_object.starttls()

(220, b'2.0.0 Ready to start TLS')

### Enter You Email and Generated App Password

In [58]:
email = os.getenv("GOOGLE_EMAIL")
password = os.getenv("GOOGLE_APP_PASSWORD")
smtp_object.login(email,password)

(235, b'2.7.0 Accepted')

In [63]:
from_address = email
to_address = os.getenv("EMAIL_TO")
subject = "libadd"
message = f"Subject: {subject}\n\n{email_json}"


result = smtp_object.sendmail(from_address, to_address, message)

if result == {}:
    print("Email sent successfully!")
else:
    print("Failed to send email to the following recipient(s):")
    print(result)

Email sent successfully!


## Working with the Other Classes

In [66]:
lib = Library()

In [68]:
lib.add_book(Book("Clean Code", "Robert C. Martin"))
lib.add_book(Book("The Pragmatic Programmer", "Andrew Hunt and David Thomas"))
lib.add_book(Book("Design Patterns: Elements of Reusable Object-Oriented Software", "Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides"))
lib.add_book(Book("Refactoring", "Martin Fowler"))
lib.add_book(Book("You Don't Know JS", "Kyle Simpson"))
lib.add_book(Magazine("Surf Today", "Kelly Slater", available=False))
lib.add_book(Magazine("National Geographic", "Various Authors"))
lib.add_book(Magazine("The Surfer's Journal", "Scott Hulet"))
lib.add_book(Newspaper("The Times", "Editorial Board", publish_date="03/2025"))
lib.add_book(Newspaper("The New York Times", "Dean Baquet", publish_date="04/2025"))

Added Clean Code by Robert C. Martin to library
Added The Pragmatic Programmer by Andrew Hunt and David Thomas to library
Added Design Patterns: Elements of Reusable Object-Oriented Software by Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides to library
Added Refactoring by Martin Fowler to library
Added You Don't Know JS by Kyle Simpson to library
Added Surf Today by Kelly Slater to library
Added National Geographic by Various Authors to library
Added The Surfer's Journal by Scott Hulet to library
Added The Times by Editorial Board to library
Added The New York Times by Dean Baquet to library


In [70]:
print(lib.list_all())

Library:
[Book] Title: Clean Code
Author: Robert C. Martin
Available: Yes

[Book] Title: The Pragmatic Programmer
Author: Andrew Hunt and David Thomas
Available: Yes

[Book] Title: Design Patterns: Elements of Reusable Object-Oriented Software
Author: Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides
Available: Yes

[Book] Title: Refactoring
Author: Martin Fowler
Available: Yes

[Book] Title: You Don't Know JS
Author: Kyle Simpson
Available: Yes

[Magazine] Title: Surf Today
Author: Kelly Slater
Available: No

[Magazine] Title: National Geographic
Author: Various Authors
Available: Yes

[Magazine] Title: The Surfer's Journal
Author: Scott Hulet
Available: Yes

[Newspaper] Title: The Times
Author: Editorial Board
Published: 03/2025
Available: Yes

[Newspaper] Title: The New York Times
Author: Dean Baquet
Published: 04/2025
Available: Yes



## Russian Book To English

In [85]:
russian_book1 = RussianBook("Преступление и наказание", "Фёдор Достоевский")
russian_book2 = RussianBook("Война и мир", "Лев Толстой")
russian_book3 = RussianBook("Мастер и Маргарита", "Михаил Булгаков")

def translated_book(book: RussianBook) -> RussianTranslatorAdapter:
    translated = RussianTranslatorAdapter(book)
    translated.display_info()
    return translated

lib.add_book(translated_book(russian_book1))
lib.add_book(translated_book(russian_book2))
lib.add_book(translated_book(russian_book3))

[Book - Translated] 'Crime and punishment' by Fedor Dostoevsky
(Original: 'Преступление и наказание' by Фёдор Достоевский) 

Added Crime and punishment by Fedor Dostoevsky to library
[Book - Translated] 'War and peace' by Leo Tolstoy
(Original: 'Война и мир' by Лев Толстой) 

Added War and peace by Leo Tolstoy to library
[Book - Translated] 'Master and Margarita' by Mikhail Bulgakov
(Original: 'Мастер и Маргарита' by Михаил Булгаков) 

Added Master and Margarita by Mikhail Bulgakov to library


In [87]:
lib.remove_book("You Don't Know JS")
lib.list_all()

Not found.


'Library:\n[RussianTranslatorAdapter] Title: Crime and punishment\nAuthor: Fedor Dostoevsky\nAvailable: Yes\n\n[RussianTranslatorAdapter] Title: War and peace\nAuthor: Leo Tolstoy\nAvailable: Yes\n\n[RussianTranslatorAdapter] Title: Master and Margarita\nAuthor: Mikhail Bulgakov\nAvailable: Yes\n'

In [89]:
print(f"{len(lib.books)}")

3


In [91]:
lib.remove_all_books()

Removed all 3 books from library


In [93]:
lib.list_all()

'Your library is empty.'